# GraphQL Endpoints

In [1]:
base_url = 'https://base.easscan.org/graphql'
base_sepolia_url = 'https://base-sepolia.easscan.org/graphql' # testnet

schemaID = '0xb763e62d940bed6f527dd82418e146a904e62a297b8fa765c9b3e1f0bc6fdd68' # OLI v1.0.0 schema

### Retrieve the latest attestations for the OLI Label Pool

In [10]:
import requests

def query_attestations(endpoint: str, schemaId: str, count: int = 10) -> dict:
    query = """
    query Attestations($take: Int, $where: AttestationWhereInput, $orderBy: [AttestationOrderByWithRelationInput!]) {
      attestations(take: $take, where: $where, orderBy: $orderBy) {
        attester
        data
        decodedDataJson
        expirationTime
        id
        ipfsHash
        isOffchain
        recipient
        refUID
        revocable
        revocationTime
        revoked
        time
        timeCreated
        txid
      }
    }
    """
    
    variables = {
        "take": count,
        "where": {
            "schemaId": {
                "equals": schemaId
            }
        },
        "orderBy": [
            {
            "timeCreated": "desc"
            }
        ]
    }
    
    headers = {
        "Content-Type": "application/json"
    }
    
    response = requests.post(endpoint, json={"query": query, "variables": variables}, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"GraphQL query failed with status code {response.status_code}: {response.text}")

# Example usage:
result = query_attestations(base_url, schemaID)
print(result)


{'data': {'attestations': [{'attester': '0xdC1d963D21C9c1bFf7b6Bea6e10080dAa9b4fc51', 'data': '{"sig":{"version":2,"domain":{"name":"EAS Attestation","version":"1.0.1","chainId":"8453","verifyingContract":"0x4200000000000000000000000000000000000021"},"primaryType":"Attest","types":{"Attest":[{"name":"version","type":"uint16"},{"name":"schema","type":"bytes32"},{"name":"recipient","type":"address"},{"name":"time","type":"uint64"},{"name":"expirationTime","type":"uint64"},{"name":"revocable","type":"bool"},{"name":"refUID","type":"bytes32"},{"name":"data","type":"bytes"},{"name":"salt","type":"bytes32"}]},"signature":{"r":"0x65c298a033ee66442ff051a8dd7f2dffc8aa170b045f9f8a5239da8c485c2f02","s":"0x3d7935f23d68f15dcba5d76d3a8d921b3688bde636b6fde6cb2b0df00681f90b","v":28},"uid":"0x72c99fe964ced352d4309b3a163b4446d722900973d507c640a9d91f1cdf332d","message":{"version":2,"schema":"0xb763e62d940bed6f527dd82418e146a904e62a297b8fa765c9b3e1f0bc6fdd68","recipient":"0x66a9893cc07d91d95644aedd05d03f9

In [11]:
import json
import pandas as pd

# create a DataFrame from the result and extract the decoded data into separate columns
df = pd.DataFrame(result['data']['attestations'])
def extract_decoded_data(json_str):
    data = json.loads(json_str)
    return {item["name"]: json.loads(item["value"]["value"]) if item["name"] == "tags_json" else item["value"]["value"] for item in data}
df = df.join(df["decodedDataJson"].apply(lambda x: pd.Series(extract_decoded_data(x))))
df

,attester,data,decodedDataJson,expirationTime,id,ipfsHash,isOffchain,recipient,refUID,revocable,revocationTime,revoked,time,timeCreated,txid,chain_id,tags_json
0,0xdC1d963D21C9c1bFf7b6Bea6e10080dAa9b4fc51,"{""sig"":{""version"":2,""domain"":{""name"":""EAS Atte...","[{""name"":""chain_id"",""type"":""string"",""signature...",0,0x72c99fe964ced352d4309b3a163b4446d722900973d5...,QmQEpPC9XXeafZ43rEfv8q1UMgwuZNiCFuT8wEQvwkTwE6,True,0x66a9893cC07D91D95644AEDD05D03f95e1dBA8Af,0x00000000000000000000000000000000000000000000...,True,0,False,1739399321,1739399337,,eip155:1,"{'is_contract': True, 'usage_category': 'dex',..."
1,0xdC1d963D21C9c1bFf7b6Bea6e10080dAa9b4fc51,0x00000000000000000000000000000000000000000000...,"[{""name"":""chain_id"",""type"":""string"",""signature...",0,0x8f79bd1f98defef4a2aaa1d417c2199432a24483a80d...,,False,0x66a9893cC07D91D95644AEDD05D03f95e1dBA8Af,0x00000000000000000000000000000000000000000000...,True,1739399367,True,1739398841,1739398844,0xaa7f44a8fc56ac8ddb0d816254353b8b8adb495af4f9...,eip155:8453,"{'is_contract': True, 'usage_category': 'dex',..."
2,0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56,"{""sig"":{""version"":2,""uid"":""0xbe80773c47fa5b26e...","[{""name"":""chain_id"",""type"":""string"",""signature...",0,0xbe80773c47fa5b26e485e23bff9cec33ee6a36133f6e...,QmcVb82JF6xQ4GchHLMEZZpcsxZfhaSc4JCLVzqJpm2buo,True,0xFffffF8244e4d4a906F9A70C13E91cB30E1Cb39A,0x00000000000000000000000000000000000000000000...,True,0,False,1739374661,1739374662,,eip155:42161,"{'is_contract': True, 'deployment_tx': '0xC6CC..."
3,0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56,"{""sig"":{""version"":2,""uid"":""0x3995fa31ee5974440...","[{""name"":""chain_id"",""type"":""string"",""signature...",0,0x3995fa31ee59744403eb80746e62cc20fd371abfd76d...,QmdggpQ7ACpmB9zCdxhVLqhJpRZNz5qwJcq2wNv52x52WS,True,0xfFff355574dA624f824Dc9C3c6F95aBD8a86a697,0x00000000000000000000000000000000000000000000...,True,0,False,1739374659,1739374660,,eip155:42161,"{'contract_name': 'sushi', 'usage_category': '..."
4,0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56,"{""sig"":{""version"":2,""uid"":""0x065cbef290a2addca...","[{""name"":""chain_id"",""type"":""string"",""signature...",0,0x065cbef290a2addca6a4a95db27a425e98cb3c3ffc3d...,QmRVSs3XLAUEVYWjyCAe1vByAj4Z3mZ723TmMfAhhn2XDw,True,0xfFfeC517af2171110417a082e4dd5558442De7f2,0x00000000000000000000000000000000000000000000...,True,0,False,1739374658,1739374658,,eip155:10,"{'contract_name': 'setprotocol_v2', 'owner_pro..."
5,0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56,"{""sig"":{""version"":2,""uid"":""0x3ebb419782bebba8b...","[{""name"":""chain_id"",""type"":""string"",""signature...",0,0x3ebb419782bebba8b223573a6bfc27daa01fb1bd4617...,QmVC8J2o2s2ffR4u1Anwai3uUtyzh4bqM3UorA8QzLbxbN,True,0xFFf6963D74B943d76DF5D1139eD045d344D21Fa6,0x00000000000000000000000000000000000000000000...,True,0,False,1739374656,1739374657,,eip155:42161,"{'is_contract': True, 'deployment_tx': '0xC020..."
6,0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56,"{""sig"":{""version"":2,""uid"":""0x05c1bfc2b012d2f24...","[{""name"":""chain_id"",""type"":""string"",""signature...",0,0x05c1bfc2b012d2f248ea04eb4cedede892a7337e8614...,QmR3z64MmMoKcywyAPqYoon8xweJyLA7itvaeQp2iYBjc2,True,0xFFeEd65FAF3bC544ecD39724689dAbb4397016da,0x00000000000000000000000000000000000000000000...,True,0,False,1739374655,1739374655,,eip155:10,{'usage_category': 'fungible_tokens'}
7,0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56,"{""sig"":{""version"":2,""uid"":""0x348b13d616d20db44...","[{""name"":""chain_id"",""type"":""string"",""signature...",0,0x348b13d616d20db44ef254616c7ea6adf134c108e6bd...,QmbdjihxP2ws8niKCYcNxCG3fnTEeNEY6m8CPuXzVjvNYp,True,0xfFeEaDe7D2d28aCe7F4691c690bc149A1FC06453,0x00000000000000000000000000000000000000000000...,True,0,False,1739374653,1739374653,,eip155:42161,"{'is_contract': True, 'deployment_tx': '0x945F..."
8,0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56,"{""sig"":{""version"":2,""uid"":""0x5c441843141670f4d...","[{""name"":""chain_id"",""type"":""string"",""signa